In [1]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
from pyemd import emd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

def combine(state, goal):
    if state is None:
        return None
    return torch.cat((state, goal), 1)

In [3]:
class miniDQN(nn.Module):

    def __init__(self, hidden):
        super(miniDQN, self).__init__()
        self.W1 = nn.Linear(8, hidden)
        self.W2 = nn.Linear(hidden, 4)
        nn.init.xavier_uniform_(self.W1.weight)
        nn.init.xavier_uniform_(self.W2.weight)

    def forward(self, x):
        x = F.relu(self.W1(x))
        return self.W2(x)

#For now, embedding on states (not state-action pairs) because we're lazy
class embeddingNN(nn.Module):

    def __init__(self, hidden, output):
        super(embeddingNN, self).__init__()
        self.W1 = nn.Linear(8, hidden)
        self.W2 = nn.Linear(hidden, output)
        nn.init.xavier_uniform_(self.W1.weight)
        nn.init.xavier_uniform_(self.W2.weight)

    def forward(self, x):
        x = F.relu(self.W1(x))
        return self.W2(x)

In [4]:
num_row = 10
num_col = 10

action_direction = [(0,1), (0,-1), (1,0), (-1,0)]
block = { "empty" : 0, "blue" : 1, "red" : 2, "goal" : 3, "wall" : 4}

reds = [(3,0)]
blues = [(0,3)]
start = (0,0)
end = (4,4)
walls = []

start_state = np.asarray(list(start) + [0,0])
end_state = np.asarray(list(end) + [1,1])


def is_done(s):
    row, col, blue, red = tuple(s)
    return blue and red and (row, col) == end

def step(s, a):
    row, col, blue, red = tuple(s)
    a_row, a_col = action_direction[a]

    row = row + a_row
    col = col + a_col

    if row < 0 or row >= num_row or col < 0 or col >= num_col or (row, col) in walls:
        return s

    blue = blue or (row, col) in blues
    red = red or (row, col) in reds

    return np.asarray((row, col, blue, red))

def get_reward(s, a, s_prime):
    if is_done(s):
        return 0.0
    return -1.0

def L1(s1, s2):
    return 1.0 * np.sum(np.abs(np.asarray(s1) - np.asarray(s2)))

In [5]:
#Very adhoc, requires no dynamics
distance_cache = {}

def distance(a, b):
    global distance_cache
    ta = tuple(a)
    tb = tuple(b)
    if (ta,tb) in distance_cache:
        return distance_cache[(ta,tb)]
    
    row_a, col_a, blue_a, red_a = ta
    row_b, col_b, blue_b, red_b = tb
    
    blue_match = blue_a == blue_b
    red_match = red_a == red_b
    
    if red_match and not blue_match:
        dist = min([L1((row_a,col_a),blue) + L1(blue, (row_b, col_b)) for blue in blues])
    elif not red_match and blue_match:
        dist = min([L1((row_a,col_a),red) + L1(red, (row_b, col_b)) for red in reds])
    elif not red_match and not blue_match:
        x = min([L1((row_a,col_a),red) + L1(red, blue) + L1(blue, (row_b, col_b)) for red, blue in zip(reds, blues)])
        y = min([L1((row_a,col_a),blue) + L1(blue, red) + L1(red, (row_b, col_b)) for red, blue in zip(reds, blues)])
        dist = min(x,y)
    else:
        dist = L1(ta, tb)
    distance_cache[(ta,tb)] = dist
    return dist

In [6]:
BATCH_SIZE = 64
GAMMA = 0.9999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 6000
TARGET_UPDATE = 50
HIDDEN_LAYER_SIZE = 15
EMBEDDING_SIZE = 10
EPISODE_LENGTH = 30
LEARNING_RATE = 0.0004

DIVERSE_EXPERT = True

num_episodes = 30000
random.seed(10)
distance_cache = {}

policy_net = miniDQN(HIDDEN_LAYER_SIZE).to(device)
target_net = miniDQN(HIDDEN_LAYER_SIZE).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

embedding_net = embeddingNN(HIDDEN_LAYER_SIZE, EMBEDDING_SIZE).to(device)
embedding_target_net = embeddingNN(HIDDEN_LAYER_SIZE, EMBEDDING_SIZE).to(device)
embedding_target_net.load_state_dict(embedding_net.state_dict())
embedding_target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr = LEARNING_RATE)
embedding_optimizer = optim.Adam(embedding_net.parameters(), lr = LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.99999)
memory = ReplayMemory(40000)
expert_memory = ReplayMemory(40000)

In [7]:
steps_done = 0

#Epsilon greedy actions
def select_action(state, goal, test = False):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if test or sample > eps_threshold:
        with torch.no_grad():
            x = torch.cat((state, goal), 1)
            return policy_net(x).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(4)]], device=device, dtype=torch.long)

#Max entropy action
def select_action_entropy(state, goal, alpha):
    x = torch.cat((state, goal), 1)
    potential = (alpha * policy_net(x).detach()).exp().squeeze()
    dist = (potential / potential.sum()).numpy()
    return torch.tensor([[np.random.choice(4, p = dist)]], device=device, dtype=torch.long)
    

def select_expert_action(state, goal):
    row, col, blue, red = tuple(state)
    action_scores = np.zeros(4)
    for a in range(4):
        #Assumes determinism
        next_state = step(state, a)
        action_scores[a] = distance(next_state, goal)
    action = np.random.choice(np.flatnonzero(action_scores == action_scores.min()))
    return torch.tensor([[action]], device=device, dtype=torch.long)    

def sample_expert_trajectory():
    state = start_state
    next_state = None

    trajectory = []
    
    for t in range(EPISODE_LENGTH):
        action = select_expert_action(state, end_state)
        
        torch_state = torch.from_numpy(state).float().unsqueeze(0)
        next_state = step(state, action.item())
        torch_next_state = torch.from_numpy(next_state).float().unsqueeze(0)
        reward = get_reward(state, action.item(), next_state)
        done = is_done(state)
        reward = torch.tensor([reward], device=device)
        if done:
            torch_next_state = None

        trajectory.append((torch_state, action, torch_next_state, reward))
        if done:
            break
        state = next_state
    return trajectory

In [8]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    #randomly learn over expert trajectories or rollout buffer
    transitions = memory.sample(BATCH_SIZE) if np.random.random() > 0.5 else expert_memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,batch.next_state)),
                                  device=device, dtype=torch.uint8)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_action_values = policy_net(state_batch).gather(1, action_batch)

    #DDQN
    policy_action_indices = policy_net(non_final_next_states).max(1)[1].detach()
    target_actions = target_net(non_final_next_states).detach()
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_actions.gather(1, policy_action_indices.unsqueeze(1)).squeeze()
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()
    return loss.item()

In [9]:
losses = [0]
torch_end = torch.from_numpy(end_state).float().unsqueeze(0)
expert_trajectory = sample_expert_trajectory()
for i in range(1000):
    if DIVERSE_EXPERT:
        expert_trajectory = sample_expert_trajectory()
    for torch_state, action, torch_next_state, reward in expert_trajectory:
        memory.push(combine(torch_state, torch_end), action, combine(torch_next_state, torch_end), reward)
        expert_memory.push(combine(torch_state, torch_end), action, combine(torch_next_state, torch_end), reward)

for i_episode in range(num_episodes):
    state = start_state
    next_state = None
    final = i_episode == num_episodes - 1
    trajectory = []
    
    if i_episode % 1000 == 0:
        scheduler.step()
    
    for t in range(EPISODE_LENGTH):
        torch_state = torch.from_numpy(state).float().unsqueeze(0)

        action = select_action(torch_state, torch_end, final)
            
        next_state = step(state, action.item())
        torch_next_state = torch.from_numpy(next_state).float().unsqueeze(0)
        reward = get_reward(state, action.item(), next_state)
        done = is_done(state)
        reward = torch.tensor([reward], device=device)

        if done:
            torch_next_state = None

        trajectory.append((torch_state, action, torch_next_state, reward))

        loss = optimize_model()
        losses.append(loss if loss is not None else 0)
        if final:
            print(state)
        if done:
            break
        state = next_state

    if i_episode % 1000 == 0:
        print(i_episode)
        print(np.mean(np.asarray(losses)))
        losses = []

    #HER updates with goal as real goal OR last state in trajectory
    for torch_state, action, torch_next_state, reward in trajectory:
        memory.push(combine(torch_state, torch_end), action, combine(torch_next_state, torch_end), reward)
        if not is_done(state):
            torch_local_goal = torch.from_numpy(state).float().unsqueeze(0)
            local_reward = torch.equal(torch_next_state, torch_local_goal)
            new_reward = torch.add(reward, local_reward)
            memory.push(combine(torch_state, torch_local_goal), action, combine(torch_next_state, torch_local_goal), new_reward)
                    
                          
    # Update the target network
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
    
        
print('Complete')




0
0.9251463913148449
1000
0.05532407117767892
2000
0.11884550596425632
3000
0.13816929148313783
4000
0.07045261357338115
5000
0.032681195712045454
6000
0.022907886051512057
7000
0.021218531527953107
8000
0.021442808433426094
9000
0.02284733171883662
10000
0.03598577695013884
11000
0.028796425871288847
12000
0.025441255420028265
13000
0.019848565537107295
14000
0.014711856696527293
15000
0.015475577203472311
16000
0.019264995628555867
17000
0.022316037964809916
18000
0.019930242033852484
19000
0.01656944545884613
20000
0.014744983269138435
21000
0.016718836453505043
22000
0.019155777290376902
23000
0.02168935358543737
24000
0.022647639601300312
25000
0.02272344616271969
26000
0.017395932136937575
27000
0.011313943188493849
28000
0.011470668027769066
29000
0.011707691411981915
[0 0 0 0]
[1 0 0 0]
[2 0 0 0]
[3 0 0 1]
[2 0 0 1]
[1 0 0 1]
[0 0 0 1]
[0 1 0 1]
[0 2 0 1]
[0 3 1 1]
[1 3 1 1]
[2 3 1 1]
[3 3 1 1]
[3 4 1 1]
[4 4 1 1]
Complete


In [14]:
def predicted_distance(state, goal):
    torch_state = torch.from_numpy(state).float().unsqueeze(0)
    torch_goal = torch.from_numpy(goal).float().unsqueeze(0)
    x = torch.cat((torch_state, torch_goal), 1)
    return -1 * policy_net(x).max(1)[0].view(1, 1).item()

def predicted_measure_ball(state, goal, n = 500):
    measure = {}
    torch_state = torch.from_numpy(state).float().unsqueeze(0)
    torch_goal = torch.from_numpy(goal).float().unsqueeze(0)
    x = torch.cat((torch_state, torch_goal), 1)
    for i in range(n):
        action = select_action_entropy(torch_state, torch_goal, alpha = 4.0)
        next_state = step(state, action.item())
        key = tuple(next_state)
        measure[key] = measure.get(key, 0) + 1.0/n
    return measure

def wasserstein(a, b, goal):
    if tuple(a) == tuple(b):
        return 0
    m_a = predicted_measure_ball(a, goal)
    m_b = predicted_measure_ball(b, goal)
    states = list(set(m_a.keys()) | set(m_b.keys()))
    
    m_a = [m_a.get(s, 0.0) for s in states]
    m_b = [m_b.get(s, 0.0) for s in states]
    dist = [[distance(np.asarray(s1),np.asarray(s2)) for s1 in states] for s2 in states]
    return emd(np.asarray(m_a), np.asarray(m_b), np.asarray(dist))


In [15]:
values = np.zeros((num_row, num_col, 2, 2))
for index in np.ndindex(num_row, num_col, 2, 2):
    values[index] = predicted_distance(np.asarray(index), end_state)
print(np.round(values[:,:,0,0], 2))
print(np.round(values[:,:,1,0], 2))
print(np.round(values[:,:,0,1], 2))
print(np.round(values[:,:,1,1], 2))

[[13.96 13.05 12.02 13.45 14.44 14.22 14.24 14.62 15.   15.38]
 [12.93 13.9  14.44 15.4  15.93 15.71 15.71 16.09 16.47 16.85]
 [11.91 12.88 13.85 15.65 17.43 17.2  17.5  18.24 18.98 19.72]
 [10.92 12.44 13.71 15.83 18.79 20.44 20.85 21.35 21.56 21.78]
 [13.66 14.18 14.71 16.73 20.86 22.95 23.07 23.48 23.28 23.08]
 [15.33 16.11 16.44 17.75 21.44 24.17 25.45 26.05 26.6  26.79]
 [15.52 15.78 16.04 17.47 21.4  24.97 27.45 28.36 29.03 29.58]
 [15.1  15.35 15.61 16.87 20.8  24.73 27.49 29.62 30.79 31.81]
 [14.67 14.93 15.19 16.27 20.2  24.13 27.49 30.48 31.75 32.92]
 [14.24 14.5  14.76 15.68 19.6  23.53 27.39 30.47 32.71 33.88]]
[[ 8.11  9.08 10.05 11.01 12.17 12.37 13.06 13.76 14.45 15.14]
 [ 7.09  8.06  9.02  9.99 12.   13.47 14.16 14.66 14.82 14.99]
 [ 6.07  7.03  8.    8.95 10.85 12.75 14.03 15.08 15.61 16.14]
 [ 5.01  6.19  7.18  8.01  9.79 12.41 14.23 15.32 16.4  17.04]
 [ 7.2   7.51  7.84  8.16 10.06 12.29 14.84 15.8  16.76 17.72]
 [ 6.77  7.03  7.29  7.54  9.46 12.94 15.72 17.42 18.3

In [16]:
def reflect_state(state):
    row, col, blue, red = tuple(state)
    return np.asarray((col, row, red, blue))

def compare(state):
    reflected = reflect_state(state)
    m_a = predicted_measure_ball(state, end_state)
    m_b = predicted_measure_ball(reflected, end_state)
    print(m_a)
    print(m_b)

compare(np.asarray((0,1,1,1)))
    

{(1, 1, 1, 1): 0.6840000000000005, (0, 2, 1, 1): 0.3140000000000002, (0, 1, 1, 1): 0.002}
{(1, 1, 1, 1): 0.25400000000000017, (2, 0, 1, 1): 0.7460000000000006}
